In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# If running type is GPU, run this cell

!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


printm()


In [1]:
# If run type is TPU, run this cell 
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

try:
  # imports the torch_xla package
  import torch_xla
  import torch_xla.core.xla_model as xm

  device = xm.xla_device()

except:
  VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION

  # imports pytorch
  import torch

  # imports the torch_xla package
  import torch_xla
  import torch_xla.core.xla_model as xm

  device = xm.xla_device()

Python library

In [2]:
import numpy as np
import pandas as pd

import re
from html.parser import HTMLParser
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report
import torch
from torch.utils.data import TensorDataset, DataLoader

import time
import datetime

import warnings
warnings.filterwarnings('ignore')

try:
  import transformers as ppb
except:
  !pip install transformers
  import transformers as ppb

In [ ]:
# If running on GPU, run this cell to enable cuda
if torch.cuda.is_available():    
  
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [3]:
def format_time(elapsed):
    # display the elapsed time when loading the data into the BERT model
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [4]:
def text_cleaning(text):
    # converting HTML character codes to ASCII code
    parser = HTMLParser()
    text = parser.unescape(text)

    text = re.sub(r'<[^>]+>', '', text)  # removing HTML tags
    text = re.sub(r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)', '', text)  # removing hash-tags
    text = re.sub('\n', ' ', text)  # remove new line
    text = re.sub('@', '', text)  # remove @ sign
    #text = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '',text)  # removing URLs
    text = re.sub(r'(?:[\ufffd]+)', '', text)  # removing special characters
    text = text.lower()

    return text

Load data

In [5]:
df = pd.read_csv('/content/gdrive/My Drive/Amazon Seller Forum/final_merged_data.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
print(df.isna().sum())
print('')

nan_row = np.where(df.isna())[0][0]
print(df.iloc[nan_row,:])

Title              0
Post Author        0
Leading Comment    1
Reply Comments     0
Category           0
Forum              0
dtype: int64

Title              Israel Cohen’s Success Story1
Post Author                          Trent-Admin
Leading Comment                              NaN
Reply Comments                                []
Category                             Misc Topics
Forum                                   Flowster
Name: 8832, dtype: object


In [6]:
df.dropna(axis=0,subset=['Leading Comment'],inplace = True)

In [7]:
comments = df['Leading Comment'].apply(lambda x:text_cleaning(x))

Load Transformer model

In [8]:
#from transformers import XLNetLMHeadModel, XLNetTokenizer

#tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
#model = XLNetLMHeadModel.from_pretrained("xlnet-base-cased",num_labels=22,
                                         #output_attentions = False, # Whether the model returns attentions weights.
                                         #output_hidden_states = False, # Whether the model returns all hidden-states.
                                         #)


# Load pretrained model/tokenizer
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-cased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

tokenize leading comments

In [24]:
label_encoder = LabelEncoder()

# Tokenize all of the sentences and map the tokens to thier word IDs.
inputs = []
attention_masks = []

# For every comment
for comment in comments:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.

    encoded_dict = tokenizer.encode_plus(
                        comment,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    inputs.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors and load them to GPU/TPU
inputs = torch.cat(inputs, dim=0)#.to(device)
attention_masks = torch.cat(attention_masks, dim=0)#.to(device)

# Encoding the labels and convert them to tensor 
labels = label_encoder.fit_transform(df['Category'])
labels = torch.tensor(labels)

Modeling 

In [25]:
batch_size = 16

dataset = TensorDataset(inputs,attention_masks)
dataloader = DataLoader(dataset,batch_size = batch_size)

time_start = time.time()

features_list = []

for step, batch in enumerate(dataloader):
  if step % 10 == 0 and not step == 0:
    elapsed = format_time(time.time()-time_start)
    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dataloader), elapsed))
  b_inpus = batch[0].to(device)
  b_attention_masks = batch[1].to(device)

  with torch.no_grad():
    last_hidden_states = model(b_inpus,attention_mask=b_attention_masks)

  last_hidden_states_cpu = last_hidden_states[0][:,0,:].cpu().numpy()
  features_list.append(last_hidden_states_cpu)


  Batch    10  of    553.    Elapsed: 0:00:04.
  Batch    20  of    553.    Elapsed: 0:00:05.
  Batch    30  of    553.    Elapsed: 0:00:06.
  Batch    40  of    553.    Elapsed: 0:00:07.
  Batch    50  of    553.    Elapsed: 0:00:07.
  Batch    60  of    553.    Elapsed: 0:00:08.
  Batch    70  of    553.    Elapsed: 0:00:09.
  Batch    80  of    553.    Elapsed: 0:00:10.
  Batch    90  of    553.    Elapsed: 0:00:10.
  Batch   100  of    553.    Elapsed: 0:00:11.
  Batch   110  of    553.    Elapsed: 0:00:12.
  Batch   120  of    553.    Elapsed: 0:00:12.
  Batch   130  of    553.    Elapsed: 0:00:13.
  Batch   140  of    553.    Elapsed: 0:00:14.
  Batch   150  of    553.    Elapsed: 0:00:15.
  Batch   160  of    553.    Elapsed: 0:00:15.
  Batch   170  of    553.    Elapsed: 0:00:16.
  Batch   180  of    553.    Elapsed: 0:00:17.
  Batch   190  of    553.    Elapsed: 0:00:18.
  Batch   200  of    553.    Elapsed: 0:00:18.
  Batch   210  of    553.    Elapsed: 0:00:19.
  Batch   220

In [26]:
all_features = []
for batch in features_list:
  for feature in batch:
    all_features.append(feature)


In [27]:
train_features, test_features, train_labels, test_labels = train_test_split(all_features, labels,test_size=0.3,random_state=42)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
pred = lr_clf.predict(test_features)

print(classification_report(test_labels,pred))


              precision    recall  f1-score   support

           0       0.64      0.75      0.69       471
           1       0.53      0.12      0.19        85
           2       0.48      0.43      0.46        88
           3       0.53      0.50      0.52       164
           4       0.00      0.00      0.00        11
           5       0.72      0.66      0.69       100
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00         4
           9       0.54      0.64      0.59       445
          10       0.51      0.42      0.46       192
          11       0.53      0.47      0.50       133
          12       0.00      0.00      0.00        17
          13       0.00      0.00      0.00         6
          14       0.51      0.46      0.49        71
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         4
          17       0.00    